In [ ]:
from joblib import dump
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

import gurobipy as gp

In [ ]:
# import my functions
import sys
%load_ext autoreload
%autoreload 2
sys.path.append("../../src")
from gurobi.machinelearning.extra.morerelu import reluOBBT, ReLUM
from gurobi.machinelearning.extra.obbt import obbt
from gurobi.machinelearning import add_predictor_constr

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)


In [ ]:
x_train = tf.reshape(tf.cast(x_train, tf.float32)/255.0, [-1, 28*28])

In [ ]:
x_test = tf.reshape(tf.cast(x_test, tf.float32)/255.0, [-1, 28*28])

In [ ]:
inputs = tf.keras.Input(shape=(28*28,))
x = tf.keras.layers.Dense(50)(inputs)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Dense(50)(x)
x = tf.keras.layers.ReLU()(x)
outputs = tf.keras.layers.Dense(10, )(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(50, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(10, )

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.out(x)


In [ ]:
model = MyModel()

In [ ]:
nn = tf.keras.models.Sequential([
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dense(10)
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
model.fit(
    x_train, y_train,
    epochs=6,
    validation_data=(x_test, y_test),
)

In [ ]:
model.save('MNIST_50-50')

In [ ]:
model = keras.models.load_model('MNIST_50-50')

In [ ]:
model.summary()

In [ ]:
first_image = x_train[13, :]
plt.imshow(tf.reshape(first_image,[28, 28]), cmap="gray")
plt.show()

In [ ]:
m = gp.Model()
epsilon = 5

x = m.addMVar((1,28*28), lb=0.0, ub=1.0, name="X")
absdiff = m.addMVar((1,28*28), lb=0, ub=1, name="dplus")
output = m.addMVar((1,10), lb=-gp.GRB.INFINITY, name="y")
m.setObjective(output[0, 3] - output[0, 6], gp.GRB.MAXIMIZE)

# Bound on the distance to example in norm-1
m.addConstr(absdiff[0, :] >= x[0, :] - first_image.numpy())
m.addConstr(absdiff[0, :] >= -x[0, :] + first_image.numpy())
m.addConstr(absdiff[0, :].sum() <= epsilon)

In [ ]:
nn2grb = add_predictor_constr(m, model, x, output)

In [ ]:
nn2grb.print_stats()

In [ ]:
obbt(nn2grb, activation=reluOBBT('both'))

In [ ]:
m.optimize()

In [ ]:
plt.imshow(x.X.reshape((28, 28)), cmap="gray")
plt.show()

In [ ]:
keras.models.Sequential?

In [ ]:
input = model.layers[0]

In [ ]:
input.

In [ ]:
type(model)

In [ ]:
isinstance(model, keras.Model)

In [ ]:
keras.engine.functional.Functional

In [ ]:
from keras.engine.functional import Functional

In [ ]:
isinstance(model.layers[2], keras.layers.ReLU)